In [1]:
#截圖、攝影機拍照，存成 Json

from PIL import ImageGrab, Image
import os
import random
from base64 import b64decode, b64encode
import pandas as pd
import numpy as np

# 開資料夾
if "temp" not in os.listdir():
    os.mkdir("temp")
    
if "data" not in os.listdir():
    os.mkdir("data")

#攝影機
#pip install opencv-python
from cv2 import VideoCapture, cvtColor, COLOR_RGB2BGR
cap = VideoCapture(0)

def getWebCamImg(): 
    ret, frame = cap.read()
    frame = cvtColor(np.array(frame), COLOR_RGB2BGR)
    return Image.fromarray(frame).resize((1280, 720), Image.ANTIALIAS)

def getScreenShot():
    screenShot = ImageGrab.grab()
    screenShot = screenShot.resize((1280, 720), Image.ANTIALIAS)
    return screenShot

def getImgB64encode(img):
    randomVal = random.getrandbits(32)
    filename = "./temp/%d.png"%randomVal
    img.save(filename)
    img_encode = b64encode(open(filename, 'rb').read())
    os.remove(filename)
    return img_encode.decode()

In [2]:
#彙整起來
def grabImgByFrame(timeFrame, recordID = "test"):
    if recordID not in os.listdir("./data/"):
        os.mkdir("./data/"+recordID)
    
    screenShot = getScreenShot()
    WebCam = getWebCamImg()

    outJson = {
        "webCam": getImgB64encode(WebCam),
        "screenShot": getImgB64encode(screenShot), 
    }

    import json
    filename = "./data/"+ recordID+"/"+ str(timeFrame) + ".json"


    with open(filename, 'w') as f:
        json.dump(outJson,f)
    
    return filename

In [3]:
grabImgByFrame("123", "test")

'./data/test/123.json'

In [4]:
import time
import threading

grabImgByFrame("0", "init")

is_screenshot = False
def screenshot(user_id, is_screenshot):
  timeframe = 0
  while True:
    grabImgByFrame(timeframe, user_id)
    time.sleep(0.7)
    timeframe += 1
    if not is_screenshot():
      break

is_screenshot = True
tmp = threading.Thread(target=screenshot, args=("test", lambda: is_screenshot))
tmp.start()
time.sleep(10)
is_screenshot = False

In [ ]:
def signal_filter(signal):
		signal = np.array(signal)
    np.random.seed(20)
    signal_length = signal.shape[0]
    signal = preprocessing.scale(signal)
    (surrogate_signal, _, _) = surrogates.iaaft(signal)
    w = modwt(signal, 'sym5', 5)
    surrogate_w = modwt(surrogate_signal, 'sym5', 5)
    for j in range(w.shape[0]):
        t_score = abs((w[j] - surrogate_w[j].mean()) / surrogate_w[j].std())
        p = 1 - stats.t.cdf(t_score, 2 * signal_length - 2, **None)
        threshold = w[j].std() * np.sqrt(2 * np.log(signal_length))
        for index in range(w[j].shape[0]):
            w[j][p[index] * 2 >= threshold] = surrogate_w[(j, index)]
        
    
    new_signal = imodwt(w, 'sym5')
    return new_signal

def get_fft_data(data):
    return np.abs(np.fft.fft(data)).astype(int)

def change_to_sequence_data(data, offset_index = (10,)):
    sequence_data = []
    for start_index in range(0, data.shape[0] - data.shape[0] % offset_index, offset_index):
        sequence_data.append(np.array(data[start_index:start_index + offset_index]))
    
    return np.array(sequence_data)


In [ ]:
# model

from keras.models import model_from_json

def load_model(model_name, model_weights_filename):
    with open("model/{}.json".format(model_name), "r") as json_file:
        model = model_from_json(json_file.read())

    model.load_weights("model/{}.h5".format(model_weights_filename))
    model.compile(loss="categorical_crossentropy",
                optimizer="adam",
                metrics=["categorical_accuracy"])
              
    return model

model = load_model("new-CNN-RNN", "new")

def predict(data):
    data = signal_filter(data)
    data = get_fft_data(data)
    data = change_to_sequence_data(data)
    predictions[user_id].append(model.predict(data))

In [ ]:
import serial

# COM 在不同裝置時會不一樣
ser = serial.Serial("COM3", 9600, timeout=2) # Establish the connection on a specific port
flag_inputdata = 0
val = ""

def SerialWrite(command):
 ser.write(bytes(command,"utf-8"))
 ser.flushInput()

def start_record_esp32():
    SerialWrite("st")
    while ser.in_waiting:
        ser.readline()
    val = ""
    flag_inputdata = 1

def end_record_esp32():
    SerialWrite("end")
    flag_inputdata = 0
    while ser.in_waiting:
        pass
    return val

def get_esp32_data():
    while True:
        if flag_inputdata == 1 and ser.in_waiting:  #收到('\n')指令旗標
            val = ser.readline()

In [ ]:
def start_screenshoot():
    global is_screenshoot
    
    is_screenshoot = True
    screen_shoot = threading.Thread(target=screenshot, args=(id, lambda: is_screenshot))
    screen_shoot.start()

# Flask API 的部分

In [6]:
import numpy as np
import pandas as pd
import json

from flask import Flask, request, abort
app = Flask(__name__)

from flask_cors import CORS
CORS(app)

import hashlib
import datetime

"""
這邊放確認是否有跟 ESP32 連線的 code
"""

# 確認連線後，開啟網頁
webbrowser.open("index.html")

# open camara
grabImgByFrame("0", "init")

# user id
user_id = ""
is_screenshot = False
predictions = {}


@app.route("/startRecord")
def start_record(local_user_id=""):
    """
    這邊放讓 ESP32 開始紀錄的 code
    """
    
    global user_id

    start_screenshoot()
    if local_user_id != "":
        user_id = local_user_id
        return "", 204

    else:
        md5 = hashlib.md5()
        md5.update(str(datetime.datetime.now()).encode())
        user_id = md5.hexdigest()
        
        return json.dump({"user_id": user_id}), 200

@app.route("/stopRecord")
def stop_record():
    """
    這邊放停止 ESP32 紀錄的 code
    """

    return "", 204

@app.route("/getPrediction")
def get_prediction(user_id):
    return json.dump({"data": [0.1, 0.8, 0.1, 0.5, 0.6]}), 200

@app.route("/test")
def test():
    return str("Your test is successful.")

@app.route("/getImg")
def getImg():
    filename = grabImgByFrame(0)
    return json.dumps({"status": "Good", "filename": filename})

#uvicorn filename:app --port 8001 --workers 1 --proxy-headers
if __name__ == "__main__":
    app.run(host = '0.0.0.0', port = 13523)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:13523/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Jul/2020 16:01:07] "GET /getImg HTTP/1.1" 200 -


## 然後
只要在瀏覽器打上 http://localhost:13520 就可以呼叫API囉<br>
比方 http://localhost:13520/test 就會回應 "Your test is successful."<br>
http://localhost:13520/getImg 就會回應 Json 圖片檔案